# Infosys Quarterly Report Analysis

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [6]:
#!pip install getpass4 chromadb llama-index llama-index-vector-stores-chroma
#!pip install llama-index-retrievers-bm25 llama-index-llms-groq llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 10.4 MB/s eta 0:00:00


In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:

import chromadb
import Stemmer
from llama_index.llms.groq import Groq
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext, Document
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.retrievers.bm25 import BM25Retriever

## Configure LLM service

In [5]:
#llm = OpenAI(temperature=0,
#             model="chatgpt-4o-latest",
#             max_tokens=500)


llm = Groq(model="llama-3.3-70b-versatile")

In [104]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [105]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

## Load data
Downloaded from

https://www.infosys.com/investors/reports-filings/quarterly-results.html

In [106]:
q1_2024 = SimpleDirectoryReader(
    input_files=["/content/converted_document.md"]
).load_data()

# Build indices

In [115]:
d = 1024

db = chromadb.PersistentClient(path="./chroma_db")
#db.delete_collection("reports")
chroma_collection = db.get_or_create_collection("reports")

In [116]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
q1_2024_index = VectorStoreIndex.from_documents(q1_2024, storage_context=storage_context)

## Build query engines

In [117]:
q1_2024__engine = q1_2024_index.as_query_engine(similarity_top_k=2)

In [118]:
# Assuming you have already built the index and query engine
retriever = q1_2024__engine.retriever
retrieved_nodes = retriever.retrieve("What is the revenue growth for the quarter?")

# Print the retrieved chunks
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Chunk {i+1} ---")
    print(node.get_text())


--- Chunk 1 ---
Key highlights:

## For the quarter ended June 30, 2023

- Revenues in CC terms grew by 4.2% YoY and by 1.0% QoQ
- Reported revenues at ` 37,933 crore, growth of 10.0% YoY
- Operating margin at 20.8%, growth of 0.7% YoY and decline of 0.2% QoQ
- Basic EPS at ` 14.37, growth of 12.4% YoY
- FCF at ` 5,749 crore, growth of 12.6% YoY; FCF conversion at 96.7% of net profit

'Q1 operating margins were resilient in an uncertain macro environment on the back of our continued focus on cost optimization. Company's rigorous operational discipline including improved productivity measures and higher utilization helped margins for the quarter' said Nilanjan Roy, CFO . 'Free Cash conversion was robust at 96.6% of net profits. Execution of strong capital allocation policy resulted in higher payouts to investors and improved ROE to 32.8%' he added.

<!-- image -->

## IFRS - INR Press Release

## 2. Client wins &amp; Testimonials

- Danske Bank recently signed a strategic collaboration

## Run queries

In [119]:
response = q1_2024__engine.query(
    "What is the revenue growth for the quarter?"
)

In [120]:
#q1_2024__engine.get_prompts()

In [121]:
print(response)

The revenue growth for the quarter is 4.2% YoY and 1.0% QoQ in constant currency terms, with reported revenues at ` 37,933 crore, representing a growth of 10.0% YoY.


In [122]:
response = q1_2024__engine.query("Which are some of the key customer wins?")

### Explore ChromaDB

In [123]:
from chromadb.config import Settings

In [124]:
collections = db.list_collections()
for c in collections:
    print(c.name)

reports
reports_hybrid


In [125]:
chroma_collection

Collection(name=reports)

In [126]:
data = chroma_collection.get(include=["embeddings", "documents", "metadatas"])
for i in range(2):
    print(f"ID: {data['ids'][i]}")
    print(f"Document: {data['documents'][i]}")
    print(f"Document: {data['embeddings'][i][0:20]}")
    print(f"Metadata: {data['metadatas'][i]}")
    print('-'*40)

ID: 373a3ec6-b0ab-4e1e-ac62-7836f7b1315c
Document: <!-- image -->

## IFRS - INR Press Release IFRS - INR Press Release

\

Solid Q1 year on year revenue growth of 4.2% at 20.8% operating margins

e

Strong large deal closures and robust deal pipeline position us well for future growth

Bengaluru, India - July 20, 2023: Infosys (NSE, BSE, NYSE:INFY), a global leader in next-generation digital services and consulting, delivered $4,617 million in Q1 revenues with year on year growth of 4.2% and sequential growth of 1.0% in constant currency. Large deal TCV for the quarter was at $2.3 billion, with net new of 56.1%. Operating margin for the quarter was stable at 20.8%. ROE improved 180 bps to 32.8%. Attrition  declined further to  17.3%. FY24 revenue guidance revised to 1.0%-3.5% and operating margin guidance retained at 20%-22%.

'We had a solid Q1 with a growth of 4.2% and large deals of $2.3 billion which helps us to set a strong foundation for future growth. Our generative AI capabili

### Hybrid Retriever

In [127]:
q1_reports = SimpleDirectoryReader(
    input_files=["/content/converted_document.md"]
).load_data()

In [128]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.storage.docstore import SimpleDocumentStore


# initialize node parser
splitter = SentenceSplitter(chunk_size=512, chunk_overlap = 20)
nodes = splitter.get_nodes_from_documents(q1_reports)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [129]:
db = chromadb.PersistentClient(path="./chroma_db")
#db.delete_collection("reports_hybrid")

In [130]:
hybrid_chroma_collection = db.get_or_create_collection("reports_hybrid")
hybrid_vector_store = ChromaVectorStore(chroma_collection=hybrid_chroma_collection)

hybrid_storage_context = StorageContext.from_defaults(
    docstore=docstore, vector_store=hybrid_vector_store
)

hybrid_index = VectorStoreIndex(nodes=nodes, storage_context=hybrid_storage_context)

In [134]:
from llama_index.core.retrievers import QueryFusionRetriever

In [135]:
hybrid_retriever = QueryFusionRetriever(
    [
        hybrid_index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=hybrid_index.docstore,
            similarity_top_k=2
        ),
    ],
    num_queries=1,
    use_async=True,
)

DEBUG:bm25s:Building index from IDs objects


In [136]:
from llama_index.core.response.notebook_utils import display_source_node

In [147]:
display_source_node?

In [152]:
nodes = hybrid_retriever.retrieve("What is CEO's message for the quarter?")
for node in nodes:
    display_source_node(node, source_length=2000)

**Node ID:** 0c2b8dbb-ae0c-44fb-aaa3-06f2a48a205d<br>**Similarity:** 1.3758165836334229<br>**Text:** <!-- image -->

## IFRS - INR Press Release IFRS - INR Press Release

\

Solid Q1 year on year revenue growth of 4.2% at 20.8% operating margins

e

Strong large deal closures and robust deal pipeline position us well for future growth

Bengaluru, India - July 20, 2023: Infosys (NSE, BSE, NYSE:INFY), a global leader in next-generation digital services and consulting, delivered $4,617 million in Q1 revenues with year on year growth of 4.2% and sequential growth of 1.0% in constant currency. Large deal TCV for the quarter was at $2.3 billion, with net new of 56.1%. Operating margin for the quarter was stable at 20.8%. ROE improved 180 bps to 32.8%. Attrition  declined further to  17.3%. FY24 revenue guidance revised to 1.0%-3.5% and operating margin guidance retained at 20%-22%.

'We had a solid Q1 with a growth of 4.2% and large deals of $2.3 billion which helps us to set a strong foundation for future growth. Our generative AI capabilities are expanding well, with 80 active client projects. Topaz, our comprehensive AI offering, is resonating well with clients. We see this being transformative for clients and enhancing our overall service portfolio' said Salil Parekh, CEO and MD. 'We have expanded the margin improvement program with a holistic set of actions for the short, medium and long-term, working on five key areas, supported by our leadership team', he added.

<!-- image -->

<!-- image -->

## Guidance for FY24:

- Revenue growth of 1.0%-3.5% in constant currency
- Operating margin of 20%-22%

## 1.<br>

**Node ID:** c8cd15b0-4852-45e4-90ed-81adef897717<br>**Similarity:** 0.6725119948387146<br>**Text:** Key highlights:

## For the quarter ended June 30, 2023

- Revenues in CC terms grew by 4.2% YoY and by 1.0% QoQ
- Reported revenues at ` 37,933 crore, growth of 10.0% YoY
- Operating margin at 20.8%, growth of 0.7% YoY and decline of 0.2% QoQ
- Basic EPS at ` 14.37, growth of 12.4% YoY
- FCF at ` 5,749 crore, growth of 12.6% YoY; FCF conversion at 96.7% of net profit

'Q1 operating margins were resilient in an uncertain macro environment on the back of our continued focus on cost optimization. Company's rigorous operational discipline including improved productivity measures and higher utilization helped margins for the quarter' said Nilanjan Roy, CFO . 'Free Cash conversion was robust at 96.6% of net profits. Execution of strong capital allocation policy resulted in higher payouts to investors and improved ROE to 32.8%' he added.

<!-- image -->

## IFRS - INR Press Release

## 2. Client wins &amp; Testimonials

- Danske Bank recently signed a strategic collaboration with Infosys to accelerate the bank's digital transformation  initiatives  with  speed  and  scale. Frans  Woelders,  Chief  Operating  Officer , Danske Bank, said, 'Our Forward '28 strategy sets clear ambitions for Danske Bank to be a leading  bank  in  a  digital  age.  This  is  backed  by  significant  investments  in  digitalization  and technology,  including  plans  to  further  develop  our  customer-facing  digital  solutions,  and modernizing our technology infrastructure to enable even better customer experiences and drive operational efficiency. We have a strong starting point, and we want to further accelerate our digital and technology transformation. We have conducted a thorough process to find a partner that can help us achieve that. Infosys has the tools, experience, and expertise to support us in accelerating our transformation using cloud and AI technologies. Given Infosys' global presence and scale, this collaboration will also give us access to wider talent pools and capabiliti...<br>

In [138]:
from llama_index.core.query_engine import RetrieverQueryEngine

hybrid_query_engine = RetrieverQueryEngine(hybrid_retriever)

In [139]:
response = hybrid_query_engine.query("What is CEO's message for the quarter?")
print(response)

The CEO, Salil Parekh, said, 'We had a solid Q1 with a growth of 4.2% and large deals of $2.3 billion which helps us to set a strong foundation for future growth. Our generative AI capabilities are expanding well, with 80 active client projects. Topaz, our comprehensive AI offering, is resonating well with clients. We see this being transformative for clients and enhancing our overall service portfolio.' He also added, 'We have expanded the margin improvement program with a holistic set of actions for the short, medium and long-term, working on five key areas, supported by our leadership team.'
